In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy.stats import ttest_ind

In [2]:
data = pd.read_csv("free_throws.csv")
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


В качеству нулевой гипотезы возьмём то, что для игроков нет разницы в результативности относительно того, в какой половине матча они совершают броски. Колонка period = тайм. Первой половиной матча будем считать значения 1 и 2, а второй — 3 и 4

In [14]:
new_df = {
    "player": [],
    "first_part_mean": [],
    "second_part_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    first_part_shots = group[group["period"] == 1.0]["shot_made"].values.tolist() + group[group["period"] == 2.0]["shot_made"].values.tolist()
    second_part_shots = group[group["period"] == 3.0]["shot_made"].values.tolist() + group[group["period"] == 4.0]["shot_made"].values.tolist()
    
    if len(first_part_shots) < 30 or len(second_part_shots) < 30:
        continue
        
    statistic, p_value = ztest(first_part_shots, second_part_shots)

    new_df["player"].append(player)
    new_df["first_part_mean"].append(np.mean(first_part_shots))
    new_df["second_part_mean"].append(np.mean(second_part_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [15]:
#Сколько игроков забили больше, чем 30 мечей
new_df.shape

(706, 4)

На результативность 661 игрока игрет роль вторая это или первая половина матча:

In [21]:
new_df[new_df.p_value > 0.05].shape

(661, 4)

In [22]:
new_df.p_value.min()

0.0002780044959317111

In [23]:
#Cравним результаты с поправкой Бонферрони и методом Холма:
from statsmodels.stats.multitest import multipletests

multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()

0

In [24]:
multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()

0

Вывод: если гипотеза проверяется на множественном количестве испытуемых, нужно использовать поправки Бонферрони и Холма, 
потому что чем больше попыток, тем больше вероятность сделать ошибку первого рода